In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv


# Introduction

This is my first attempt at Kaggling and I thought I should practise my time-series skills on a (hopefully) real dataset, hence my participation in this competition.

In this notebook, I'm simply just going to warm-up with very basic analysis and perhaps build a baseline SARIMAX model.

In [2]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("fivethirtyeight")
plt.rcParams["figure.figsize"] = [12, 7]
plt.rcParams["figure.dpi"] = 100
plt.rcParams["figure.facecolor"] = "white"

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')

In [3]:
from statsmodels.tsa.seasonal import seasonal_decompose, STL
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf

In [4]:
# GLOBAL VARIABLES
DATA_PATH = "/kaggle/input/m5-forecasting-accuracy/"

WALMART_COLOURS = ["#0072CE", "#B4B4B3", "#79B8F3", "#FDB927", "#F7941D", "#4CB748", "#2E3192"]
DIVERGENT_COLOUR_GRADIENT = ["#e2f1fc", "#b9dcfa", "#8cc7f7", "#5eb1f3", "#39a0f1", "#0691ef"]
sns.set_palette(WALMART_COLOURS)

In [5]:
# data paths
sales_path = os.path.join(DATA_PATH, "sales_train_evaluation.csv")
calendar_path = os.path.join(DATA_PATH, "calendar.csv")
prices_path = os.path.join(DATA_PATH, "sell_prices.csv")

In [6]:
sales = pd.read_csv(sales_path)
calendar = pd.read_csv(calendar_path)
prices = pd.read_csv(prices_path)

# Functions

In [7]:
def get_df_memory(df):
  return np.round(df.memory_usage().sum()/(1024*1024),1)

def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df 

In [8]:
# def plot_time_series(data, title):
#   fig, ax = plt.subplots(figsize=(15, 6), layout="constrained")
#   _ = sns.lineplot(data=data, ax=ax)
#   _ = fig.suptitle(title, fontsize=16)

def plot_time_series(data, title):
    fig = px.line(data, title=title, color_discrete_sequence=WALMART_COLOURS)
#     fig.update_layout(height=500, width=800)
    fig.show()

In [9]:
def plot_bar_graph(data, title):
    fig = px.bar(data, title=title, color_discrete_sequence=WALMART_COLOURS)
    fig.show()

In [10]:
def add_stl_plot(fig, res, legend):
    """Add 3 plots from a second STL fit"""
    axs = fig.get_axes()
    comps = ["trend", "seasonal", "resid"]
    for ax, comp in zip(axs[1:], comps):
        series = getattr(res, comp)
        if comp == "resid":
            ax.plot(series, marker="o", linestyle="none")
        else:
            ax.plot(series)
            if comp == "trend":
                ax.legend(legend, frameon=False)

In [11]:
def view_decomposition(df):
  res_robust = STL(df, period=52, robust=True).fit()
  fig = res_robust.plot()
  res_non_robust = STL(df, period=52, robust=False).fit()
  add_stl_plot(fig, res_non_robust, ["Robust", "Non-robust"])

# Downcasting

In [12]:
sales_bd = get_df_memory(sales)
calendar_bd = get_df_memory(calendar)
prices_bd = get_df_memory(prices)

In [13]:
sales = downcast(sales)
prices = downcast(prices)
calendar = downcast(calendar)

In [14]:
sales_ad = get_df_memory(sales)
calendar_ad = get_df_memory(calendar)
prices_ad = get_df_memory(prices)

In [15]:
memory = {
  'DataFrame':['sales','calendar','prices'],
  'Before downcasting':[sales_bd,calendar_bd,prices_bd],
  'After downcasting':[sales_ad,calendar_ad,prices_ad]
}

memory = pd.DataFrame(memory)
memory = pd.melt(memory, id_vars='DataFrame', var_name='Status', value_name='Memory (MB)')
memory.sort_values('Memory (MB)',inplace=True)
fig = px.bar(memory, x='DataFrame', y='Memory (MB)', color='Status', color_discrete_sequence=WALMART_COLOURS, barmode='group', text='Memory (MB)')
fig.update_traces(texttemplate='%{text} MB', textposition='outside')
fig.update_layout(template='seaborn', title='Effect of Downcasting')
fig.show()

## Understanding The Data

Now that we've downcasted the data(a useful thing that I saw quite a number of other Kagglers in this competition doing), we can now try and wrap our heads around what's in the data. The M5 dataset, generously made available by Walmart, involves the unit sales of various products sold in the USA, organized in the form of **grouped time series**. We have 3 049 products classified into 3 product categories (Hobbies, Foods and Household). We then have 7 product departments, under which these categories are aggregated.
These products are sold in 10 stores across 3 states: California(CA), Texas(TA) and Wisconsin(WI).

We can visualize this distribution as follows:

In [16]:
sales_group = sales.groupby(['state_id','store_id','cat_id','dept_id'],as_index=False)['item_id'].count().dropna()
sales_group.head()

,state_id,store_id,cat_id,dept_id,item_id
0,CA,CA_1,FOODS,FOODS_1,216
1,CA,CA_1,FOODS,FOODS_2,398
2,CA,CA_1,FOODS,FOODS_3,823
3,CA,CA_1,FOODS,HOBBIES_1,0
4,CA,CA_1,FOODS,HOBBIES_2,0


In [17]:
sales_group = sales_group[sales_group['item_id'] > 0].reset_index(drop=True) # removing zero-valued rows
sales_group['USA'] = 'United States of America'
sales_group.rename(columns={'state_id':'State','store_id':'Store','cat_id':'Category','dept_id':'Department','item_id':'Count'},inplace=True)
fig = px.treemap(
  sales_group, 
  path=['USA', 'State', 'Store', 'Category', 'Department'], 
  values='Count',
  color='Count',
  color_continuous_scale= DIVERGENT_COLOUR_GRADIENT,
  title="Walmart's Item Distribution"
)
fig.update_layout(template='seaborn')
fig.show()

We learn from the above graph that items are distributed in a similar manner across states, stores and product departments; all 3 states under analysis have the same number of products across stores and product categories. A curious question might be: do they really sell the same products? For now, I don't think it's worthwhile to answer this question.

The above graph also suggests the logic in which we can explore the data. +3000 products is a really a lot and thus needs systematic approach to analysis.

## Overall Sales Analysis

In [18]:
mlt_sales = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold').dropna()

In [19]:
mlt_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [20]:
mlt_sales = pd.merge(mlt_sales, calendar[['date', 'd']], on='d', how='left')
mlt_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29


In [21]:
mlt_sales['date'] = pd.to_datetime(mlt_sales['date'])
mlt_sales = mlt_sales.set_index('date')

In [22]:
daily_total_sales = mlt_sales.groupby('date')['sold'].sum()
daily_total_sales.head()

date
2011-01-29    32631
2011-01-30    31749
2011-01-31    23783
2011-02-01    25412
2011-02-02    19146
Name: sold, dtype: int64

In [23]:
plot_time_series(daily_total_sales, "Total Walmart's Daily Sales")

In [24]:
daily_total_sales[daily_total_sales.values < 100]

date
2011-12-25    13
2012-12-25    11
2013-12-25    20
2014-12-25    20
2015-12-25    14
Name: sold, dtype: int64

Oh! So it turns out that those apparent dips comes from Christmas days of each year. I initally though they're 0s, which would mean the stores close on Christmas, however, we see from above that they're just too low(almost 20 sales that day). I therefore think we should add `is_chrismas` as a feature for prediction.

Looking at the graph overall, we learn that there's practically no visible trend. If we were to be petty, I think it would be suitable to model it as a cubic polynomial of the form $y = a(x - h)^3 + k$. This simply just comes from my familiarity of how functions look like.

What I do notice as non-negligible is the yearly seasonality which shows - except in 2011 - sales take off slowly in January, peaks somewhere after June(mid-year) and begins to dip afterwards...

Nonetheless, this graph is rather noisy and will need a little smoothing to clearly see these patterns. To this end, let's try some upsampling, perhaps by week or month.

In [25]:
mlt_sales['is_christmas'] = ((mlt_sales.index.month == 12) & (mlt_sales.index.day == 25)).astype(int)

In [26]:
weekly_sales = mlt_sales[['sold']].resample('W').sum()

In [27]:
plot_time_series(weekly_sales.iloc[1:], "Total Walmart's Weekly Sales")

I've excluded the initial week to make the patterns more visible since it contains an outlier (significantly low sales). This is to ensure a clearer depiction of underlying patterns by mitigating the impact of the outlier on the overall scale of the graph.

In [28]:
monthly_sales = mlt_sales[['sold']].resample('1M').sum()

In [29]:
plot_time_series(monthly_sales.iloc[1:], "Total Walmart's Monthly Sales")

### Effects of Smoothening

Looking at the daily sales, it was very hard to even seen the seasonality and trend; however, with weekly and monthly sales these have become more apparent. Since I'd also like to create a sales report(not only build a machine learning model), I think I shoudl elaborate more on the above graph by also showing year-by-year percentage changes.

In [30]:
yearly_sales = mlt_sales[['sold']].resample('Y').sum()
yearly_sales['year'] = yearly_sales.index.year
yearly_sales.set_index('year', inplace=True)
yearly_sales['pct_change'] = yearly_sales["sold"].pct_change() * 100

# adding some aesthetics, as this is going into a report
yearly_sales['colour'] = yearly_sales['pct_change'].apply(lambda x: WALMART_COLOURS[1] if x >= 0 else WALMART_COLOURS[0])
yearly_sales.head()

,sold,pct_change,colour
year,,,
2011,8856585,NaN,#0072CE
2012,12061837,36.190608,#B4B4B3
2013,13135753,8.903420,#B4B4B3
2014,13089776,-0.350014,#0072CE
2015,13800811,5.431988,#B4B4B3


In [31]:
fig = px.bar(
    data_frame=yearly_sales.iloc[1:], # excluding nan in this case
    y='pct_change',
    title="Year-by-Year Sales Percentage Change",
    color='colour',
    color_discrete_sequence=WALMART_COLOURS
)

fig.show()

Consequences of a cubic function continue to show themselves as there's the highest increase from 2011 to 2012, a plateau until 2015 and a mejor dip from 2015 to 2016. \
**Important:** The major dip in 2016 May was because the month of May was when the data was last recorded; we see from daily and weekly data that the sales didn't necessarily reduce, rather not recorded.

It's important to note that, while 2016 seemed to have decrease(in terms of year-by-year sales percentage change), this year had not yet ended.

## Looking at Sales by State, Store & Department

In [32]:
mlt_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,is_christmas
date,,,,,,,,,
2011-01-29,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,0
2011-01-29,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,0
2011-01-29,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,0
2011-01-29,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,0
2011-01-29,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,0


In [33]:
monthly_state_sales = (mlt_sales
                       .groupby(['state_id', pd.Grouper(freq='M')])['sold']
                       .sum()).unstack(level=0)
monthly_state_sales.head()

state_id,CA,TX,WI
date,,,
2011-01-31,38108,25846,24209
2011-02-28,318101,212015,196259
2011-03-31,339678,225162,198727
2011-04-30,331004,220681,186028
2011-05-31,321566,221620,176376


In [34]:
plot_time_series(monthly_state_sales[1:], "Monthly Sales by State")

We learn from `state monthly sales` the California is the most important market as it - by far - has the most sales. This may be due to the higher standards of living due to places like San Francisco where major tech companies are situated, thus, increasing the purchasing power in the area. \
There's not much difference between Texas and Wisconsin, I suppose this is because they have a similarly budding business environments i.e. their standards of living are comparable. What's interesting however, is that between 2011 and 2013, Texas had been outperforming Wisconsin until somewhat competed equally from 2013 to 2016, then in 2016, tables turned as Wisconsin began to outperform Texas. ***It would be interesting to find out why?***

Anyway, overall, we do see an upward trend in all states, even though California is quite dominant. Also, **August** seems to be the magic month.

**Questions to answer(keep them in mind):**
- [ ] Why did Texas outperform Wisconsi prior to 2013 but see on opposite effect post 2015?
- [ ] What happened between April 2013 to November 2013?

Moving forward, let's learn about our products and store and department levels

In [35]:
mlt_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,is_christmas
date,,,,,,,,,
2011-01-29,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,0
2011-01-29,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,0
2011-01-29,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,0
2011-01-29,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,0
2011-01-29,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,0


In [36]:
# monthly_store_sales = mlt_sales.groupby(['state_id', 'store_id', pd.Grouper(freq='M')])['sold'].sum().reset_index()
# monthly_store_sales = monthly_store_sales.pivot_table(values='sold', index='date', columns=['store_id'], fill_value=0).astype(int)
# monthly_store_sales.head()

In [37]:
monthly_store_sales = mlt_sales.groupby(['state_id', 'store_id', pd.Grouper(freq='M')])['sold'].sum().reset_index()
monthly_store_sales_filtered = monthly_store_sales[~((monthly_store_sales['date'].dt.year == 2011) & (monthly_store_sales['date'].dt.month == 1))]
filtered_sales = monthly_store_sales_filtered[monthly_store_sales_filtered['sold'] != 0]
filtered_sales.head()

,state_id,store_id,date,sold
1,CA,CA_1,2011-02-28,89434
2,CA,CA_1,2011-03-31,93468
3,CA,CA_1,2011-04-30,91553
4,CA,CA_1,2011-05-31,87988
5,CA,CA_1,2011-06-30,90376


In [38]:
fig = px.line(
    filtered_sales, x='date', 
    y='sold', 
    color='state_id',
    line_group='store_id', 
    labels={'sold': 'Sales'},
    title='Monthly Sales per Store per State',
    color_discrete_sequence=[WALMART_COLOURS[0], WALMART_COLOURS[2], WALMART_COLOURS[4], WALMART_COLOURS[6]]
)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Sales',
    legend_title='State',
    hovermode='x unified'
)

fig.show()

## Store Analysis

There are 4 stores in California, 3 of which perform at about the same rate as stores in Texas and Wisconsin; there's only one store that seems to do far better than all of them. It's also noteworthy that CA_4 is almost not affected by seasonality like other CA stores which seem to follow the same seasonal pattern.
If we only view Texan stores, we notice that store 1 dominated the other 2 stores from 2011 to 2014(especially between 2012 and 2014), afterwards its performance dropped until store 2 caught up with it. In Wisconsin, there's not a whole lot of growth. We only see growth in 2012 where store 2 experience a game-changer in June followed by store 3 in November. Both of these stores never looked back after this increase, however, they never increased any further.

**Question to keep in mind:** 
- [ ] CA_3 went through a bit of a rough path after May 2014, can the data we have explain this?
- [ ] Is there a reason why TX_1 was so dominant between 2012 and 2014?
- [ ] What happened in 2012(June and November) in Wisconsin? What caused such a huge increse in sales in stores 2 and 3?

In general, we do see that the seasonal effect is only more apparent in Californian stores.

Moving forward, let's see if we can find anything interesting from more granular levels i.e. product categories and product departments.


## Product Categories and Departments

To get even more granular, we may ask: Which products drive the most sales in each state?

In [39]:
mlt_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,is_christmas
date,,,,,,,,,
2011-01-29,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,0
2011-01-29,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,0
2011-01-29,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,0
2011-01-29,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,0
2011-01-29,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,0


In [40]:
prod_cats = (
    mlt_sales
    .groupby(['cat_id', pd.Grouper(freq='M')])['sold']
    .sum()
    .reset_index()
    .pivot_table(values='sold', index='date', columns='cat_id')
    .astype(int)
)
prod_cats.head()

cat_id,FOODS,HOBBIES,HOUSEHOLD
date,,,
2011-01-31,63110,9803,15250
2011-02-28,520250,77362,128763
2011-03-31,534914,83691,144962
2011-04-30,509568,86199,141946
2011-05-31,491753,87413,140396


In [58]:
plot_time_series(prod_cats[1:-1], "Monthly Sales by Product Categories")

**FINDINGS**

*Food Products* \
Starting from May 2011, we notice that the sales have been on an updward trajectory. Perhaps this suggests there may have been a successful intervention that may have increased the sales baseline from somewhere around or below 500k monthly sales(prior to March 2011) to 700k monthly sales in March 2012. Speaking of 2012, we notice that 2012 was a beginning of a yearly seasonal pattern where sales of food products started low at the beginning of the year, increase unti they peak somewhere around August and then immediately decrease until the cycle repeats the following year.

*Hobbies Products* \
Hobbies products' sales can be explained into 3 periods 2011-2013, 2013 - 2015 and 2015 - 2016. In the first period, monthly sales consistently flactuated well between 80k - 90k from 2011 Jan to 2012 June where they suddenly tanked to 70k until the end of the year. The sales picked up from this all-time low(which was in November of 2012) to 100k monthly sales; a journey that took 5 months until April of 2013. 100k monthly sales then became average performance until the end of 2014. 2015 brought a new era where sales never went below 100k monthly sales until April of 2016. \
**In summary, it seems there's a 2-year plan for how Hobbies products are going to perform, after-which a re-evaluation is made for the next 2 years.**

*Household Products** \
Similar to hobbies products, there seems to be a strategy applied in order to manage sales as we see a period of consistent performance from 2012 to 2015, after which we see a noticeable increase in monthly sales.

**In conclusion** we learn that the products under consideration have their periods where - perhaps - the management employs a certain strategy for some time and then improves on it after. This also means there's a general upward trend of monthly sales(unsurprisingly).

**FURTHER INVESTIGATIONS**

Understanding the high-level sales progression of each product category, we'll now be diving even deeper into the actual products on how they're doing i.e. which are the most bought.

In [92]:
item_counts = pd.DataFrame(
    mlt_sales[['cat_id', 'dept_id', 'item_id']]
    .drop_duplicates(ignore_index=True)
    .groupby(['cat_id', 'dept_id'])['item_id']
    .count()
    .reset_index()
)
item_counts = item_counts[item_counts['item_id'] != 0]
item_counts = item_counts.reset_index(drop=True)
item_counts.sort_values(by='item_id', ascending=False, inplace=True)

In [95]:
fig = px.bar(
    item_counts,
    x='dept_id',
    y='item_id',
    labels={
        'dept_id': 'Department',
        'item_id': 'Number of Products'
    },
    title='Number of Products per Department',
    color_discrete_sequence=WALMART_COLOURS
)

fig.show()

In [78]:
mlt_sales['year'] = mlt_sales.index.year
total_sales = pd.DataFrame(
    mlt_sales
    .groupby(['year', 'cat_id', 'dept_id'])['sold']
    .sum()
    .reset_index()
)
total_sales = total_sales[total_sales['sold'] != 0]
total_sales = total_sales.reset_index(drop=True)
total_sales

,year,cat_id,dept_id,sold
0,2011,FOODS,FOODS_1,610070
1,2011,FOODS,FOODS_2,1139010
2,2011,FOODS,FOODS_3,4509784
3,2011,HOBBIES,HOBBIES_1,875816
4,2011,HOBBIES,HOBBIES_2,63335
5,2011,HOUSEHOLD,HOUSEHOLD_1,1217866
6,2011,HOUSEHOLD,HOUSEHOLD_2,440704
7,2012,FOODS,FOODS_1,1011192
8,2012,FOODS,FOODS_2,1318851
9,2012,FOODS,FOODS_3,6330225


In [81]:
total_sales['perc_contribs'] = (
    total_sales
    .groupby('year')['sold']
    .transform(lambda x: x / x.sum() * 100)
)
total_sales

,year,cat_id,dept_id,sold,perc_contribs
0,2011,FOODS,FOODS_1,610070,6.888321
1,2011,FOODS,FOODS_2,1139010,12.860600
2,2011,FOODS,FOODS_3,4509784,50.920123
3,2011,HOBBIES,HOBBIES_1,875816,9.888868
4,2011,HOBBIES,HOBBIES_2,63335,0.715118
5,2011,HOUSEHOLD,HOUSEHOLD_1,1217866,13.750966
6,2011,HOUSEHOLD,HOUSEHOLD_2,440704,4.976004
7,2012,FOODS,FOODS_1,1011192,8.383400
8,2012,FOODS,FOODS_2,1318851,10.934081
9,2012,FOODS,FOODS_3,6330225,52.481434


In [87]:
fig = px.bar(
    total_sales,
    x='year',
    y='perc_contribs',
    color='dept_id',
    facet_col='cat_id',
    labels={
        'perc_contribs': 'Percentage Contributions',
        'cat_id': 'Category',
        'dept_id': 'Department'
    },
    title='Yearly Percentage Contribution of each Department per Product Category',
    color_discrete_sequence=WALMART_COLOURS
)

fig.show()

So we learn that, amongst other things, the `FOODS` category contributes about $70\%$ of overall sales, which is not a far-fetched finding because this category also has the highest number of products as per the graph above. Looking at the deparments, we learn that `Department 3` under the `FOODS` category drives about $50\%$ of overall sales and `Category 2` under `HOBBIES` category has the least effect sales around $1\%$.

In [82]:
total_sales.groupby('year')['perc_contribs'].sum()

year
2011    100.0
2012    100.0
2013    100.0
2014    100.0
2015    100.0
2016    100.0
Name: perc_contribs, dtype: float64

In [214]:
EXCL_COLS = ['2011-01-31', '2016-05-31']

def sort_product_sales_by_sum(sales_df, ascending=False):
    row_sums = sales_df.sum(axis=1)
    sales_df = sales_df.loc[row_sums.sort_values(ascending=ascending).index]
    return sales_df

def get_products_sales_per_dept(dept_name):
    dept_sales = (
        mlt_sales[mlt_sales['dept_id'] == dept_name][['item_id', 'sold']]
        .groupby(['item_id', pd.Grouper(freq='1M')])['sold']
        .sum()
        .reset_index()
        .pivot_table(index='item_id', columns='date', values='sold', fill_value=0)
    )

    dept_sales = dept_sales.loc[dept_sales.sum(axis=1) != 0]
    return dept_sales

In [215]:
food3_sales = get_products_sales_per_dept('FOODS_3')
foods3_sales_sorted = sort_product_sales_by_sum(food3_sales)

food3_sales_sorted.head()

date,2011-02-28,2011-03-31,2011-04-30,2011-05-31,2011-06-30,2011-07-31,2011-08-31,2011-09-30,2011-10-31,2011-11-30,...,2015-07-31,2015-08-31,2015-09-30,2015-10-31,2015-11-30,2015-12-31,2016-01-31,2016-02-29,2016-03-31,2016-04-30
item_id,,,,,,,,,,,,,,,,,,,,,
FOODS_3_090,6306,0,0,0,0,0,0,6169,30156,29115,...,19051,17231,16841,17375,17333,929,13459,15796,16921,16926
FOODS_3_586,13154,14416,15067,15437,15797,17017,19722,17292,18026,17153,...,16012,16993,13416,14543,13733,14603,11367,12222,12556,12415
FOODS_3_252,7187,7838,8562,9077,10255,11445,13079,10520,9590,8238,...,12137,13461,10004,9559,8154,8802,6659,8159,8522,8698
FOODS_3_555,7863,8909,9139,9202,9539,10719,12284,10373,10441,9776,...,7394,8149,6833,7419,6805,7524,6025,6647,7009,7221
FOODS_3_587,9979,9084,10215,9197,7798,8063,8825,7619,8079,8126,...,8155,7570,7484,7907,6864,6839,8101,8204,7624,7149


In [216]:
food3_sales_sorted.tail()

date,2011-02-28,2011-03-31,2011-04-30,2011-05-31,2011-06-30,2011-07-31,2011-08-31,2011-09-30,2011-10-31,2011-11-30,...,2015-07-31,2015-08-31,2015-09-30,2015-10-31,2015-11-30,2015-12-31,2016-01-31,2016-02-29,2016-03-31,2016-04-30
item_id,,,,,,,,,,,,,,,,,,,,,
FOODS_3_260,0,0,0,0,0,0,0,0,0,0,...,91,152,144,120,89,66,84,97,87,123
FOODS_3_255,0,0,0,0,0,0,0,0,0,0,...,79,104,106,101,66,57,87,73,109,125
FOODS_3_472,0,0,0,0,0,0,0,0,0,0,...,37,43,75,94,70,43,96,78,70,119
FOODS_3_171,0,0,0,0,0,0,0,0,0,0,...,43,44,43,60,69,51,43,34,19,26
FOODS_3_220,0,0,0,0,0,0,0,0,0,0,...,1,8,1,4,0,17,61,83,86,67


In [217]:
top_food3_sales = food3_sales_sorted[:36]
top_food3_sales.head()

date,2011-02-28,2011-03-31,2011-04-30,2011-05-31,2011-06-30,2011-07-31,2011-08-31,2011-09-30,2011-10-31,2011-11-30,...,2015-07-31,2015-08-31,2015-09-30,2015-10-31,2015-11-30,2015-12-31,2016-01-31,2016-02-29,2016-03-31,2016-04-30
item_id,,,,,,,,,,,,,,,,,,,,,
FOODS_3_090,6306,0,0,0,0,0,0,6169,30156,29115,...,19051,17231,16841,17375,17333,929,13459,15796,16921,16926
FOODS_3_586,13154,14416,15067,15437,15797,17017,19722,17292,18026,17153,...,16012,16993,13416,14543,13733,14603,11367,12222,12556,12415
FOODS_3_252,7187,7838,8562,9077,10255,11445,13079,10520,9590,8238,...,12137,13461,10004,9559,8154,8802,6659,8159,8522,8698
FOODS_3_555,7863,8909,9139,9202,9539,10719,12284,10373,10441,9776,...,7394,8149,6833,7419,6805,7524,6025,6647,7009,7221
FOODS_3_587,9979,9084,10215,9197,7798,8063,8825,7619,8079,8126,...,8155,7570,7484,7907,6864,6839,8101,8204,7624,7149


Visualizing the data below, I notice that there a products with outliear sales(very few thought) which mess up the scale of the colour gradients, so let's just remove products with monthly sales above 500 because, from just eye-balling the values, we have 2 values that are 599 and 750.

In [218]:
#252 on top; 503 at the bottom
resorted_food3_sales = sort_product_sales_by_sum(top_food3_sales, True)
resorted_food3_sales[:-2].head()

date,2011-02-28,2011-03-31,2011-04-30,2011-05-31,2011-06-30,2011-07-31,2011-08-31,2011-09-30,2011-10-31,2011-11-30,...,2015-07-31,2015-08-31,2015-09-30,2015-10-31,2015-11-30,2015-12-31,2016-01-31,2016-02-29,2016-03-31,2016-04-30
item_id,,,,,,,,,,,,,,,,,,,,,
FOODS_3_281,3534,3507,1453,0,0,0,0,943,112,13,...,2036,1939,1859,107,46,26,2066,2286,2719,2492
FOODS_3_811,0,0,0,0,0,0,0,0,0,0,...,4659,1850,3339,3608,2970,3253,2898,4074,4452,4943
FOODS_3_547,2889,2804,2047,2919,3425,3686,3845,3773,3489,3173,...,2527,2888,1918,2032,2578,2421,2530,1926,2132,1775
FOODS_3_030,4464,1928,905,3,1,2,2727,3199,4151,3988,...,2517,2120,2037,811,100,55,1606,2705,2879,2697
FOODS_3_007,3166,2516,2627,2454,2334,3079,3294,3402,2579,3161,...,3461,3624,3036,2269,2099,2665,1938,1138,1337,2508


In [219]:
fig = go.Figure(data=go.Heatmap(
    z=resorted_food3_sales[:-2].values,
    x=resorted_food3_sales.columns,
    colorscale='Blues'
))

fig.update_layout(
    title='Monthly Sales of Food Products: Top 40 Best Peforming products',
    xaxis_title='Date',
    yaxis_title='Department',
    height=700
)

fig.show()

After a few iterations on the number of products to include in the above graph, we learn that, while Department 3 of the Food Products has the highest number of products, only 36 products are the ones which drive the sales the most. The pattern we saw on the monthly Food product sales is more apparent with these products. We unfortunately don't have the names of these products to understand further, what they are so that we can learn something about our buyers. This doesn't nully the fact that the number of products might be one of the most - if not the most - contributing factors to why the Food category dominates sales like this.

**Next thing we can do is:** \
Check if all products have always been there since 2011 as I suppose this might've been one of the reasons behind the `HOBBIES'` trajectory. \
**Simplifying assumption:** If an item hasn't been sold(yet), then we'll consider it unavailable, regardless of whether or not it's there. We'll only consider it's availability if it's been sold before.

In [252]:
hobbies1_sales = get_products_sales_per_dept('HOBBIES_1')
hobbies1_sales.drop(columns=EXCL_COLS, inplace=True)
hobbies1_sales_bin = hobbies1_sales.applymap(lambda x: 1 if x != 0 else 0)
hobbies1_sales_bin = sort_product_sales_by_sum(hobbies1_sales_bin, False)
hobbies1_sales.head()

date,2011-02-28,2011-03-31,2011-04-30,2011-05-31,2011-06-30,2011-07-31,2011-08-31,2011-09-30,2011-10-31,2011-11-30,...,2015-07-31,2015-08-31,2015-09-30,2015-10-31,2015-11-30,2015-12-31,2016-01-31,2016-02-29,2016-03-31,2016-04-30
item_id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001,0,0,0,0,0,0,0,0,0,0,...,167,112,152,149,172,146,159,154,185,151
HOBBIES_1_002,0,28,0,0,43,75,80,91,96,92,...,84,90,70,87,78,175,58,80,80,72
HOBBIES_1_003,0,0,0,0,0,0,0,0,0,0,...,66,68,55,59,71,145,57,69,78,93
HOBBIES_1_004,388,417,490,651,665,656,518,515,533,534,...,551,490,480,466,505,512,586,510,481,508
HOBBIES_1_005,0,0,0,232,331,344,349,376,435,357,...,249,315,301,347,343,316,331,288,356,262


In [253]:
prod_depts = (
    mlt_sales
    .groupby(['dept_id', pd.Grouper(freq='M')])['sold']
    .sum()
    .reset_index()
    .pivot_table(values='sold', index='date', columns='dept_id')
    .astype(int)
)
prod_depts.head()

dept_id,FOODS_1,FOODS_2,FOODS_3,HOBBIES_1,HOBBIES_2,HOUSEHOLD_1,HOUSEHOLD_2
date,,,,,,,
2011-01-31,6216,11477,45417,9279,524,10790,4460
2011-02-28,54155,97919,368176,73316,4046,90773,37990
2011-03-31,53527,101792,379595,78140,5551,105264,39698
2011-04-30,50678,95555,363335,80455,5744,103027,38919
2011-05-31,47358,95489,348906,81096,6317,99905,40491


In [254]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=prod_depts[1:-1].reset_index()['date'], y=prod_depts[1:-1]['HOBBIES_1'], name="Sales", marker_color=WALMART_COLOURS[3]),
    secondary_y=True,
)

fig.add_trace(
    go.Heatmap(
        z=hobbies1_sales_bin.values,
        x=hobbies1_sales_bin.columns,
        colorscale='GnBu'
    ),
    secondary_y=False,
)

fig.update_layout(
    title='Comparing HOBBIES sales by their Availability',
    xaxis_title='Date',
    yaxis_title='Sales',
)

fig.show()

The relationship between the availability of HOBBIES_1 items in stores and their total monthly sales appears to be disconnected. Despite a consistent and steady increase in the number of items in store, the sales patterns seem to exhibit a degree of independence, following unrelated phases.

In [43]:
plot_time_series(prod_depts[['FOODS_1', 'FOODS_2', 'FOODS_3']][1:], "Food Products: Monthly Sales by Product Departments")

In [44]:
plot_time_series(prod_depts[['HOBBIES_1', 'HOBBIES_2']][1:], "Hobbies Products: Monthly Sales by Product Departments")

In [45]:
plot_time_series(prod_depts[['HOUSEHOLD_1', 'HOUSEHOLD_2']][1:], "Household Products: Monthly Sales by Product Departments")

# Analysis on Price Sensitivity

In this section of the notebook, I'll be trying to investigate how customers are price sensitive. It's not going to be a comprehensive investigation as we have no customer data, we'll just use the data we have.

First I'd like to how often prices change. It's almost impractical to analyse how prices have changed for all products, so I need determine a reasonable way to aggregate prices.

In [46]:
prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812


I think it'll be easier to analyse prices based on actual dates instead of `wm_yr_wk`, so let's add date to the prices df by merging calender and prices on wm_yr_wk.

In [47]:
calendar_dates = calendar.reset_index(inplace=False)[['date', 'wm_yr_wk']]
calendar_dates.head()

,date,wm_yr_wk
0,2011-01-29,11101
1,2011-01-30,11101
2,2011-01-31,11101
3,2011-02-01,11101
4,2011-02-02,11101


In [48]:
prices = (pd.merge(prices, calendar_dates, on='wm_yr_wk', how='left')).set_index('date')
prices.head()

,store_id,item_id,wm_yr_wk,sell_price
date,,,,
2013-07-13,CA_1,HOBBIES_1_001,11325,9.578125
2013-07-14,CA_1,HOBBIES_1_001,11325,9.578125
2013-07-15,CA_1,HOBBIES_1_001,11325,9.578125
2013-07-16,CA_1,HOBBIES_1_001,11325,9.578125
2013-07-17,CA_1,HOBBIES_1_001,11325,9.578125


I think a way to get a sense of overall percentage change is to get a monthly percentage change of selling price per product over a month.

In [49]:
prices['pct_change'] = prices.groupby('item_id')['sell_price'].pct_change()
prices.head()

,store_id,item_id,wm_yr_wk,sell_price,pct_change
date,,,,,
2013-07-13,CA_1,HOBBIES_1_001,11325,9.578125,NaN
2013-07-14,CA_1,HOBBIES_1_001,11325,9.578125,0.0
2013-07-15,CA_1,HOBBIES_1_001,11325,9.578125,0.0
2013-07-16,CA_1,HOBBIES_1_001,11325,9.578125,0.0
2013-07-17,CA_1,HOBBIES_1_001,11325,9.578125,0.0


In [50]:
# Monthly sales
monthly_pct_changes = prices.resample('1M')['pct_change'].sum()
monthly_pct_changes.head()

date
2011-01-31   -329.383301
2011-02-28    -31.251953
2011-03-31     30.054199
2011-04-30     72.147461
2011-05-31    -32.344238
Freq: M, Name: pct_change, dtype: float32

In [51]:
plot_time_series(monthly_pct_changes[1:], "Selling Price: Overall Monthly Percentage Changes")

In [52]:
merged_df = pd.merge(monthly_pct_changes[1:], monthly_sales.iloc[1:], on='date', how='outer').reset_index()
merged_df.head()

,date,pct_change,sold
0,2011-02-28,-31.251953,726375.0
1,2011-03-31,30.054199,763567.0
2,2011-04-30,72.147461,737713.0
3,2011-05-31,-32.344238,719562.0
4,2011-06-30,20.223633,753380.0


In [53]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=merged_df['date'], y=merged_df['pct_change'], name="Price Changes", marker_color=WALMART_COLOURS[3]),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=merged_df['date'], y=merged_df['sold'], name="Sales", marker_color=WALMART_COLOURS[0]),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Monthly Sums: Comparing Sales and Selling Prices"
)

# Set axes titles
fig.update_xaxes(title_text="Yearly Months")
fig.update_yaxes(title_text="Price Changes", secondary_y=False)
fig.update_yaxes(title_text="Sales", secondary_y=True)

fig.show()

In [54]:
merged_df.corr()

,date,pct_change,sold
date,1.000000,0.170025,0.819264
pct_change,0.170025,1.000000,0.203576
sold,0.819264,0.203576,1.000000


**Conclusion on Price Sensitivity**

Looking at all the previous graphs above, there's no indication that perhaps the sales are price sensitive. Of course we are looking at aggregated values, but so far I have no reason to believe the sales affect the sales.

# Event Analysis

In [55]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [56]:
calendar['event_name_1'].unique()

[NaN, 'SuperBowl', 'ValentinesDay', 'PresidentsDay', 'LentStart', ..., 'Chanukah End', 'NewYear', 'OrthodoxChristmas', 'MartinLutherKingDay', 'Easter']
Length: 31
Categories (30, object): ['Chanukah End', 'Christmas', 'Cinco De Mayo', 'ColumbusDay', ..., 'SuperBowl', 'Thanksgiving', 'ValentinesDay', 'VeteransDay']

In [57]:
calendar['event_name_2'].unique()

[NaN, 'Easter', 'Cinco De Mayo', 'OrthodoxEaster', 'Father's day']
Categories (4, object): ['Cinco De Mayo', 'Easter', 'Father's day', 'OrthodoxEaster']